In [1]:
import pandas as pd
import numpy as np
from io import StringIO
import csv
import re
import fasttext


from sklearn.metrics import recall_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score

import nltk
from nltk.tokenize import word_tokenize

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV

In [2]:
stop_words = nltk.corpus.stopwords.words("russian")

In [3]:
def text_prepear2(text):

    text = re.sub(r'\S+#\S+;',' ', str(text))

    text = re.sub(r'#\S+',' ', text)

    text = re.sub(r'\S+#',' ', text)

    text = re.sub(r'_+',' ', text)

    text = re.sub(r'\W', ' ', text)

    text = re.sub(r'\s+[a-zA-Zа-яА-ЯёЁ]\s+', ' ', text)

    text = re.sub(r'\^[a-zA-Zа-яА-ЯёЁ]\s+', ' ', text)

    text = re.sub(r'\s+', ' ', text)

    text = text.lower()
    
    tokens = text.split()
    
    tokens = [word for word in tokens if word not in stop_words]
    
    preprocessed_text = ' '.join(tokens)
    
    return preprocessed_text

In [5]:
df1 = pd.read_csv('text_damaged_7059.csv')
df1['text_prep'] = df1['0'].apply(text_prepear2)
df1['target'] = np.ones((len(df1)))

In [6]:
df1.head(10)

,0,text_prep,target
0,Продаю уазик. Хороший торг. На полном ходу,продаю уазик хороший торг полном ходу,1.0
1,Прочитайте перед звонком! Продаётся авто в хор...,прочитайте звонком продаётся авто хорошем сост...,1.0
2,"Авто брал год назад, первый владелец в России....",авто брал год назад первый владелец россии езд...,1.0
3,Арест. Всё вопросы по тел.,арест всё вопросы тел,1.0
4,"Продаетса: уаз хантер, дизель, двигатель андор...",продаетса уаз хантер дизель двигатель андория ...,1.0
5,"На бодром ходу,уедет куда угодно,на пружинах, ...",бодром ходу уедет угодно пружинах обмен торг к...,1.0
6,"перед дтп был сделан кап ремонт ходовой,год на...",дтп сделан кап ремонт ходовой год назад менял ...,1.0
7,"Здравствуйте \nРядная 6-ка находу , но нужно в...",здравствуйте рядная 6 ка находу нужно вложения...,1.0
8,Словил клин коленвала\nКонтракций мотор - 35-4...,словил клин коленвала контракций мотор 35 40к ...,1.0
9,"Продам авто! битый , находу.",продам авто битый находу,1.0


In [7]:
df2 = pd.read_csv('text_no_damaged_37615.csv')
df2['text_prep'] = df2['0'].apply(text_prepear2)
df2['target'] = np.zeros((len(df2)))

In [8]:
df2.head(10)

,0,text_prep,target
0,"В продаже Lada Granta в последнем поколении, в...",продаже lada granta последнем поколении удобно...,0.0
1,В продаже отличная Lada Vesta.\nСостояние авто...,продаже отличная lada vesta состояние автомоби...,0.0
2,В продаже автомобиль Volkswagen Polo. Состояни...,продаже автомобиль volkswagen polo состояние а...,0.0
3,В продаже Hyundai Santa Fe. 1 владельца по ПТС...,продаже hyundai santa fe 1 владельца птс cосто...,0.0
4,&#x26a1; 30 000 СКИДКА В КРЕДИТ\n&#x26a1; 20 0...,30 000 скидка кредит 20 000 скидка обмене феде...,0.0
5,"Собственник. По технической части исправен, в ...",собственник технической части исправен дтп фак...,0.0
6,"Кондиционер,Сигнализация,MP3,Легкосплавные дис...",кондиционер сигнализация mp3 легкосплавные дис...,0.0
7,В продаже надежный Mazda CX -7. Двигатель 2.3 ...,продаже надежный mazda cx 7 двигатель 2 3 ориг...,0.0
8,Данный автомобиль прошел комплексную диагности...,данный автомобиль прошел комплексную диагности...,0.0
9,Данный автомобиль прошел комплексную диагности...,данный автомобиль прошел комплексную диагности...,0.0


In [9]:
df = pd.concat([df1, df2], ignore_index=True)
df.drop('0', axis=1, inplace=True)
df['text_prep'] = df['text_prep'].apply(lambda x: 'for drop' if len(x)<5 else x)
df.drop(df[df['text_prep'] == 'for drop'].index, inplace=True)
df.index = range(len(df))

In [11]:
df.head(10)

,text_prep,target
0,продаю уазик хороший торг полном ходу,1.0
1,прочитайте звонком продаётся авто хорошем сост...,1.0
2,авто брал год назад первый владелец россии езд...,1.0
3,арест всё вопросы тел,1.0
4,продаетса уаз хантер дизель двигатель андория ...,1.0
5,бодром ходу уедет угодно пружинах обмен торг к...,1.0
6,дтп сделан кап ремонт ходовой год назад менял ...,1.0
7,здравствуйте рядная 6 ка находу нужно вложения...,1.0
8,словил клин коленвала контракций мотор 35 40к ...,1.0
9,продам авто битый находу,1.0


In [12]:
model = fasttext.load_model("fil2.bin")

In [13]:
x_mvec = []
for i in range(len(df)):
    x_mvec.append(np.mean([model.get_word_vector(x) for x in df['text_prep'][i].split()], axis=0))

In [14]:
dfm = pd.DataFrame(x_mvec)

In [15]:
dfm['target'] = df['target']

In [16]:
from sklearn.model_selection import train_test_split
X = dfm.drop(['target'], axis=1)
y = dfm['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [19]:
skfold = StratifiedKFold(n_splits=3, shuffle=True)

In [20]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(class_weight='balanced', max_iter=1000)

scores = cross_validate(logreg, X, y, cv=skfold, 
                        scoring=('accuracy', 'roc_auc', 'recall', 'precision', 'f1'))
df_sc = pd.DataFrame(list(scores.values())[2:], 
             index=['accuracy','roc_auc','recall', 'precision', 'f1'], 
             columns=['fold 1', 'fold 2', 'fold 3'])
df_sc['mean'] = list(df_sc.T.mean())
df_sc = round(df_sc, 3) 
df_sc

,fold 1,fold 2,fold 3,mean
accuracy,0.867,0.874,0.870,0.870
roc_auc,0.943,0.947,0.945,0.945
recall,0.884,0.896,0.890,0.890
precision,0.547,0.563,0.554,0.555
f1,0.676,0.691,0.683,0.684


In [21]:
from sklearn.neighbors import KNeighborsClassifier
kn = KNeighborsClassifier()

scores = cross_validate(kn, X, y, cv=skfold, 
                        scoring=('accuracy', 'roc_auc', 'recall', 'precision', 'f1'))
df_sc = pd.DataFrame(list(scores.values())[2:], 
             index=['accuracy','roc_auc','recall', 'precision', 'f1'], 
             columns=['fold 1', 'fold 2', 'fold 3'])
df_sc['mean'] = list(df_sc.T.mean())
df_sc = round(df_sc, 3) 
df_sc

,fold 1,fold 2,fold 3,mean
accuracy,0.898,0.894,0.898,0.897
roc_auc,0.882,0.882,0.885,0.883
recall,0.501,0.512,0.517,0.510
precision,0.769,0.734,0.762,0.755
f1,0.607,0.603,0.616,0.609


In [22]:
from sklearn.svm import SVC
svc = SVC(class_weight='balanced')

scores = cross_validate(svc, X, y, cv=skfold, 
                        scoring=('accuracy', 'roc_auc', 'recall', 'precision', 'f1'))
df_sc = pd.DataFrame(list(scores.values())[2:], 
             index=['accuracy','roc_auc','recall', 'precision', 'f1'], 
             columns=['fold 1', 'fold 2', 'fold 3'])
df_sc['mean'] = list(df_sc.T.mean())
df_sc = round(df_sc, 3) 
df_sc

,fold 1,fold 2,fold 3,mean
accuracy,0.876,0.873,0.880,0.876
roc_auc,0.950,0.951,0.953,0.952
recall,0.899,0.899,0.905,0.901
precision,0.567,0.560,0.576,0.568
f1,0.696,0.690,0.704,0.696


In [26]:
import lightgbm as lgb
from lightgbm import LGBMClassifier
lgbm = LGBMClassifier(class_weight='balanced')

scores = cross_validate(lgbm, X, y, cv=skfold, 
                        scoring=('accuracy', 'roc_auc', 'recall', 'precision', 'f1'))
df_sc = pd.DataFrame(list(scores.values())[2:], 
             index=['accuracy','roc_auc','recall', 'precision', 'f1'], 
             columns=['fold 1', 'fold 2', 'fold 3'])
df_sc['mean'] = list(df_sc.T.mean())
df_sc = round(df_sc, 3) 
df_sc

,fold 1,fold 2,fold 3,mean
accuracy,0.889,0.888,0.887,0.888
roc_auc,0.952,0.949,0.948,0.949
recall,0.862,0.852,0.847,0.854
precision,0.602,0.603,0.599,0.602
f1,0.709,0.706,0.702,0.706


In [ ]:
# ##########

In [27]:
%%time
model = SVC()
param = dict(C=np.linspace(0.001, 2, 7), 
             kernel=['linear', 'rbf'], 
             random_state=[4210], 
             class_weight=['balanced'])
clf = GridSearchCV(model, param, cv=StratifiedKFold(n_splits=3, shuffle=True), scoring='roc_auc')
clf.fit(X, y)
print('roc_auc: {:.4}, with best: {}'.format(clf.best_score_, clf.best_params_))

roc_auc: 0.9515, with best: {'C': 1.6668333333333332, 'class_weight': 'balanced', 'kernel': 'rbf', 'random_state': 4210}
Wall time: 36min 6s


In [28]:
from sklearn.svm import SVC
svc = SVC(class_weight='balanced', C=1.66)

scores = cross_validate(svc, X, y, cv=skfold, 
                        scoring=('accuracy', 'roc_auc', 'recall', 'precision', 'f1'))
df_sc = pd.DataFrame(list(scores.values())[2:], 
             index=['accuracy','roc_auc','recall', 'precision', 'f1'], 
             columns=['fold 1', 'fold 2', 'fold 3'])
df_sc['mean'] = list(df_sc.T.mean())
df_sc = round(df_sc, 3) 
df_sc

,fold 1,fold 2,fold 3,mean
accuracy,0.877,0.873,0.881,0.877
roc_auc,0.952,0.951,0.951,0.951
recall,0.899,0.895,0.902,0.899
precision,0.569,0.561,0.578,0.569
f1,0.697,0.690,0.705,0.697


In [32]:
%%time
model = LGBMClassifier()
param = dict(
    max_depth=[-1,2,3],
    n_estimators=[100, 500],
    min_child_weight=np.linspace(0.001, 1, 5),
    random_state=[4210],
    class_weight=['balanced'])
clf = GridSearchCV(model, param, cv=StratifiedKFold(n_splits=3, shuffle=True), scoring='roc_auc')
clf.fit(X, y)
print('roc_auc: {:.4}, with best: {}'.format(clf.best_score_, clf.best_params_))

roc_auc: 0.9507, with best: {'class_weight': 'balanced', 'max_depth': -1, 'min_child_weight': 0.25075, 'n_estimators': 500, 'random_state': 4210}
Wall time: 2min 18s


In [33]:
import lightgbm as lgb
from lightgbm import LGBMClassifier
lgbm = LGBMClassifier(class_weight='balanced', min_child_weight=0.250, n_estimators=500)

scores = cross_validate(lgbm, X, y, cv=skfold, 
                        scoring=('accuracy', 'roc_auc', 'recall', 'precision', 'f1'))
df_sc = pd.DataFrame(list(scores.values())[2:], 
             index=['accuracy','roc_auc','recall', 'precision', 'f1'], 
             columns=['fold 1', 'fold 2', 'fold 3'])
df_sc['mean'] = list(df_sc.T.mean())
df_sc = round(df_sc, 3) 
df_sc

,fold 1,fold 2,fold 3,mean
accuracy,0.909,0.909,0.909,0.909
roc_auc,0.950,0.949,0.950,0.950
recall,0.768,0.751,0.760,0.760
precision,0.689,0.696,0.693,0.693
f1,0.726,0.722,0.725,0.725


In [ ]:
# #######

In [30]:
best_model = SVC(class_weight='balanced', C=1.66)
best_model.fit(X, y)

SVC(C=1.66, class_weight='balanced')

In [31]:
import joblib
joblib.dump(best_model, "model_damage.pkl") 

['model_damage.pkl']